In [47]:
#Coluna Testing
using JuMP, BlockDecomposition, Coluna, MathOptInterface, Gurobi
const BD = BlockDecomposition;
# const MOI = MathOptInterface;
nb_machines = 2
nb_jobs = 3 #30
# c = [12.7 22.5 8.9 20.8 13.6 12.4 24.8 19.1 11.5 17.4 24.7 6.8 21.7 14.3 10.5 15.2 14.3 12.6 9.2 20.8 11.7 17.3 9.2 20.3 11.4 6.2 13.8 10.0 20.9 20.6;  19.1 24.8 24.4 23.6 16.1 20.6 15.0 9.5 7.9 11.3 22.6 8.0 21.5 14.7 23.2 19.7 19.5 7.2 6.4 23.2 8.1 13.6 24.6 15.6 22.3 8.8 19.1 18.4 22.9 8.0;  18.6 14.1 22.7 9.9 24.2 24.5 20.8 12.9 17.7 11.9 18.7 10.1 9.1 8.9 7.7 16.6 8.3 15.9 24.3 18.6 21.1 7.5 16.8 20.9 8.9 15.2 15.7 12.7 20.8 10.4;  13.1 16.2 16.8 16.7 9.0 16.9 17.9 12.1 17.5 22.0 19.9 14.6 18.2 19.6 24.2 12.9 11.3 7.5 6.5 11.3 7.8 13.8 20.7 16.8 23.6 19.1 16.8 19.3 12.5 11.0]
# w = [61 70 57 82 51 74 98 64 86 80 69 79 60 76 78 71 50 99 92 83 53 91 68 61 63 97 91 77 68 80; 50 57 61 83 81 79 63 99 82 59 83 91 59 99 91 75 66 100 69 60 87 98 78 62 90 89 67 87 65 100; 91 81 66 63 59 81 87 90 65 55 57 68 92 91 86 74 80 89 95 57 55 96 77 60 55 57 56 67 81 52;  62 79 73 60 75 66 68 99 69 60 56 100 67 68 54 66 50 56 70 56 72 62 85 70 100 57 96 69 65 50]
# Q = [1020 1460 1530 1190]
c = [12.7 22.5 8.9; 19.1 24.8 24.4]
w = [61 70 57; 50 57 61]
Q = [1020 1460 1530] 

coluna = optimizer_with_attributes(
    Coluna.Optimizer,
    "params" => Coluna.Params(
        solver = Coluna.Algorithm.TreeSearchAlgorithm() # default BCP
    ),
    "default_optimizer" => Gurobi.Optimizer # GLPK for the master & the subproblems
)

@axis(M, 1:nb_machines)
J = 1:nb_jobs

model = BlockModel(coluna)
@variable(model, x[m in M, j in J], Bin)
@constraint(model, cov[j in J], sum(x[m, j] for m in M) >= 1)
@constraint(model, knp[m in M], sum(w[m, j] * x[m, j] for j in J) <= Q[m])
@objective(model, Min, sum(c[m, j] * x[m, j] for m in M, j in J))
callback_called = false
@dantzig_wolfe_decomposition(model, decomposition, M)

master = getmaster(decomposition)
subproblems = getsubproblems(decomposition)

specify!.(subproblems, lower_multiplicity = 0, upper_multiplicity = 1)
function my_callback_function(cb_data)
#     callback_called = true
#     x_val = callback_value.(Ref(cb_data), x)
    # Retrieve the index of the subproblem (it will be one of the values in M)
    cur_machine = BD.callback_spid(cbdata, model)

    # Retrieve reduced costs of subproblem variables
    red_costs = [BD.callback_reduced_cost(cbdata, x[cur_machine, j]) for j in J]
#     cur_machine = BD.callback_spid(cbdata, model)
# #     println("Has Duals ", has_duals(cur_machine))
# #     println(x_val)
#     if x_val[1,1] + x_val[1,2] > 1
#         con = @build_constraint(
#             x[1,1] + x[1,2] <= 1
#         )
#         println("Adding $(con)")
#         MOI.submit(model, MOI.LazyConstraint(cb_data), con)
#     end
#     println("PRICE: ", shadow_price(con))
#     return con
end
con = MOI.set(model, MOI.LazyConstraintCallback(), my_callback_function)
println(model)
optimize!(model)
println("Obj Val = ", getobjectivevalue(model))
@show callback_called

for i = 1:nb_jobs
    Z = value.(x[i,:]).data  # j-th position is equal to 1 if job j assigned to machine i
    println("Assignment for machine ", i,": ", findall(Z.>0))
end

Min 12.7 x[1,1] + 22.5 x[1,2] + 8.9 x[1,3] + 19.1 x[2,1] + 24.8 x[2,2] + 24.4 x[2,3]
Subject to
 cov[1] : x[1,1] + x[2,1] >= 1.0
 cov[2] : x[1,2] + x[2,2] >= 1.0
 cov[3] : x[1,3] + x[2,3] >= 1.0
 knp[1] : 61 x[1,1] + 70 x[1,2] + 57 x[1,3] <= 1020.0
 knp[2] : 50 x[2,1] + 57 x[2,2] + 61 x[2,3] <= 1460.0
 x[1,1] binary
 x[2,1] binary
 x[1,2] binary
 x[2,2] binary
 x[1,3] binary
 x[2,3] binary

Coluna
Version 0.3.8 | 2021-04-15 | https://github.com/atoptima/Coluna.jl
***************************************************************************************
**** BaB tree root node
**** Local DB = -Inf, global bounds : [ -Inf , Inf ], time = 0.00 sec.
***************************************************************************************
  <it=  1> <et= 0.00> <mst= 0.00> <sp= 0.00> <cols= 2> <al= 0.00> <DB=-29887.6000> <mlp=30000.0000> <PB=Inf>


┌ Warning: No initial primal bound and no cost for global artificial variables.
│ Cost of global artificial variables set to 100000.0
└ @ Coluna C:\Users\din\.julia\packages\Coluna\xXsE0\src\optimize.jl:20
┌ Warning: No initial primal bound and no cost for local artificial variables.
│ Cost of local artificial variables set to 10000.0
└ @ Coluna C:\Users\din\.julia\packages\Coluna\xXsE0\src\optimize.jl:33


UndefVarError: UndefVarError: cbdata not defined

In [1]:
using Coluna, BlockDecomposition, JuMP, MathOptInterface, Gurobi;
const BD = BlockDecomposition;
const MOI = MathOptInterface;
nb_machines = 4;
nb_jobs = 30;

c = [12.7 22.5 8.9 20.8 13.6 12.4 24.8 19.1 11.5 17.4 24.7 6.8 21.7 14.3 10.5 15.2 14.3 12.6 9.2 20.8 11.7 17.3 9.2 20.3 11.4 6.2 13.8 10.0 20.9 20.6;  19.1 24.8 24.4 23.6 16.1 20.6 15.0 9.5 7.9 11.3 22.6 8.0 21.5 14.7 23.2 19.7 19.5 7.2 6.4 23.2 8.1 13.6 24.6 15.6 22.3 8.8 19.1 18.4 22.9 8.0;  18.6 14.1 22.7 9.9 24.2 24.5 20.8 12.9 17.7 11.9 18.7 10.1 9.1 8.9 7.7 16.6 8.3 15.9 24.3 18.6 21.1 7.5 16.8 20.9 8.9 15.2 15.7 12.7 20.8 10.4;  13.1 16.2 16.8 16.7 9.0 16.9 17.9 12.1 17.5 22.0 19.9 14.6 18.2 19.6 24.2 12.9 11.3 7.5 6.5 11.3 7.8 13.8 20.7 16.8 23.6 19.1 16.8 19.3 12.5 11.0];
w = [61 70 57 82 51 74 98 64 86 80 69 79 60 76 78 71 50 99 92 83 53 91 68 61 63 97 91 77 68 80; 50 57 61 83 81 79 63 99 82 59 83 91 59 99 91 75 66 100 69 60 87 98 78 62 90 89 67 87 65 100; 91 81 66 63 59 81 87 90 65 55 57 68 92 91 86 74 80 89 95 57 55 96 77 60 55 57 56 67 81 52;  62 79 73 60 75 66 68 99 69 60 56 100 67 68 54 66 50 56 70 56 72 62 85 70 100 57 96 69 65 50];
Q = [1020 1460 1530 1190];
coluna = optimizer_with_attributes(
    Coluna.Optimizer,
    "params" => Coluna.Params(
        solver = Coluna.Algorithm.TreeSearchAlgorithm() # default BCP
    ),
    "default_optimizer" => Gurobi.Optimizer # GLPK for the master & the subproblems
);
model = BlockModel(coluna);

nb_machines = 1;
@axis(M, 1:nb_machines);
J = 1:nb_jobs;

@variable(model, x[m in M, j in J], Bin);
@constraint(model, cov[j in J], sum(x[m,j] for m in M) == 1);
@objective(model, Min, sum(c[m,j]*x[m,j] for m in M, j in J));
@dantzig_wolfe_decomposition(model, dwdec, M);
using KnapsackLib;
function my_pricing_callback(cbdata)
    # Retrieve the index of the subproblem (it will be one of the values in M)
    cur_machine = BD.callback_spid(cbdata, model)

    # Retrieve reduced costs of subproblem variables
    red_costs = [BD.callback_reduced_cost(cbdata, x[cur_machine, j]) for j in J]
    println("RD = ", red_costs)
    # Keep jobs with negative reduced costs
    selected_jobs = Int[]
    selected_costs = Int[]
    for j in J
        if red_costs[j] <= 0.0
            push!(selected_jobs, j)
            # scale reduced costs to int
            push!(selected_costs, -round(Int, 10000 * red_costs[j]))
        end
    end

    if sum(w[cur_machine, selected_jobs]) <= Q[cur_machine]
        # Don't need to run the algorithm,
        # all jobs with negative reduced cost are assigned to the machine
        jobs_assigned_to_cur_machine = selected_jobs
    else
        # Run the algorithm
        items = [KnapItem(w,c) for (w,c) in zip(w[cur_machine, selected_jobs], selected_costs)]
        data = KnapData(Q[cur_machine], items)

        # Solve the knapsack with the algorithm from KnapsackLib
        _, jobs_assigned_to_cur_machine = solveKnapExpCore(data)
    end

    # Create the solution (send only variables with non-zero values)
    sol_vars = [x[cur_machine, j] for j in jobs_assigned_to_cur_machine]
    sol_vals = [1.0 for _ in jobs_assigned_to_cur_machine]
    sol_cost = sum(red_costs[j] for j in jobs_assigned_to_cur_machine)
    println("sol_vars = ", sol_vars)
    println("sol_vals = ", sol_vals)
    println("sol_cost = ", sol_cost)
    # Submit the solution to the subproblem to Coluna
    MOI.submit(model, BD.PricingSolution(cbdata), sol_cost, sol_vars, sol_vals)
    return
end
subproblems = BD.getsubproblems(dwdec);
BD.specify!.(subproblems, lower_multiplicity = 0, solver = my_pricing_callback);
optimize!(model);
println(objective_value(model))

Coluna
Version 0.3.10 | 2021-09-07 | https://github.com/atoptima/Coluna.jl
***************************************************************************************
**** BaB tree root node
**** Local DB = -Inf, global bounds : [ -Inf , Inf ], time = 4.91 sec.
***************************************************************************************
RD = [12.7, -9977.5, -9991.1, 20.8, 13.6, 12.4, 24.8, -9980.9, 11.5, 17.4, -9975.3, -9993.2, 21.7, 14.3, -9989.5, -9984.8, 14.3, 12.6, 9.2, -9979.2, 11.7, 17.3, -9990.8, 20.3, 11.4, -9993.8, 13.8, 10.0, 20.9, 20.6]
sol_vars = VariableRef[x[1,2], x[1,3], x[1,8], x[1,11], x[1,12], x[1,15], x[1,16], x[1,20], x[1,23], x[1,26]]
sol_vals = [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
sol_cost = -99856.1
  <it=  1> <et=12.75> <mst= 1.85> <sp= 2.66> <cols= 1> <al= 0.00> <DB=  143.9000> <mlp=100000.0000> <PB=Inf>
RD = [12.7, 10022.5, 10008.9, 20.8, -9986.4, 12.4, 24.8, 9875.2, -9988.5, -9982.6, 10024.7, 10006.8, 21.7, 14.3, -9989.5, -9984.8, -9841.8

┌ Info: Column generation algorithm has converged.
└ @ Coluna.Algorithm C:\Users\din\.julia\dev\Coluna\src\Algorithm\colgen.jl:797


Node is already conquered. No children will be generated.

┌ Info: Phase one determines infeasibility.
└ @ Coluna.Algorithm C:\Users\din\.julia\dev\Coluna\src\Algorithm\colgen.jl:793



 ──────────────────────────────────────────────────────────────────────────────
                                       Time                   Allocations      
                               ──────────────────────   ───────────────────────
       Tot / % measured:          317548s / 0.00%           8.76GiB / 21.8%    

 Section               ncalls     time   %tot     avg     alloc   %tot      avg
 ──────────────────────────────────────────────────────────────────────────────
 Coluna                     1    13.5s   100%   13.5s   1.91GiB  100%   1.91GiB
   SolveLpForm              3    1.32s  9.79%   439ms    123MiB  6.29%  40.9MiB
   Inserting columns        3    279ms  2.08%  93.1ms   21.7MiB  1.11%  7.22MiB
   Cleanup columns          3    135ms  1.00%  44.9ms   6.51MiB  0.33%  2.17MiB
   Store records            1   88.3ms  0.66%  88.3ms   9.84MiB  0.50%  9.84MiB
   Update Lagrangia...      3   52.5ms  0.39%  17.5ms   3.48MiB  0.18%  1.16MiB
   Update reduced c...      3   48.5ms

┌ Info: Terminated
└ @ Coluna C:\Users\din\.julia\dev\Coluna\src\optimize.jl:63
┌ Info: Primal bound: Inf
└ @ Coluna C:\Users\din\.julia\dev\Coluna\src\optimize.jl:64


Inf


┌ Info: Dual bound: Inf
└ @ Coluna C:\Users\din\.julia\dev\Coluna\src\optimize.jl:65


In [3]:
using

UndefVarError: UndefVarError: Pkg not defined